In [1]:
import numpy as np
import pandas as pd
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models import FastText
import warnings
warnings.filterwarnings('ignore')

In [2]:
thresholds=[4,5,6,7,8]

In [3]:
#calculation of cosine similarity
def cosine_sim(vec1,vec2):
    return 10*(vec1.dot(vec2)/(norm(vec1)*norm(vec2)))

# Preprosessing

1. Word similarity dataset

In [4]:
#load word similarity file along with ground truth values
file=pd.read_csv('hindi_word_similarity.txt',names=["word1","word2","similarity"],sep=',',header=None)
words = file[["word1","word2"]]
#ground_truth = file[["similarity"]]

In [6]:
for threshold in thresholds:
    pred_column = "Ground_truth_"+str(threshold)
    file[pred_column] = np.where(file["similarity"] >= threshold, 1, 0)

In [7]:
file

,word1,word2,similarity,Ground_truth_4,Ground_truth_5,Ground_truth_6,Ground_truth_7,Ground_truth_8
0,मोहब्बत,सेक्स,6.8,1,1,1,0,0
1,बाघ,बिल्ली,7.0,1,1,1,1,0
2,किताब,कागज़,7.6,1,1,1,1,0
3,कंप्यूटर,कीबोर्ड,7.6,1,1,1,1,0
4,कंप्यूटर,इंटरनेट,8.0,1,1,1,1,1
...,...,...,...,...,...,...,...,...
60,मनोविज्ञान,चिकित्सक,7.0,1,1,1,1,0
61,मनोविज्ञान,मन,7.4,1,1,1,1,0
62,मनोविज्ञान,स्वास्थ्य,6.2,1,1,1,0,0
63,मनोविज्ञान,विज्ञान,7.0,1,1,1,1,0


# GloVe

In [8]:
#loading glove word embedding for 50 and 100 dimensions
g50 = pd.read_csv("./hi/50/glove/hi-d50-glove.txt",sep=" ",header=None)
g100= pd.read_csv("./hi/100/glove/hi-d100-glove.txt",sep=" ",header=None)
#g200=pd.read_csv("./hi/200/glove/hi-d200-glove.txt",sep=" ",header=None)
#g300=pd.read_csv("./hi/300/glove/hi-d300-glove.txt",sep=" ",header=None)

In [9]:
#extracting glove word embeddings for given words
def glove_data(df,word1,word2):
    temp1 = df.loc[df[0] == word1]
    temp1 =  temp1.T
    temp1 = list(temp1.iloc[1:,0])
   
    temp2 = df.loc[df[0] == word2]
    temp2 =  temp2.T
    temp2 = list(temp2.iloc[1:,0])
    
    vec1 =np.array(temp1)
    vec2 =np.array(temp2)
    return vec1,vec2

In [10]:
g50_sim,g100_sim,g200_sim,g300_sim =[],[],[],[]

for index, row in words.iterrows(): #iterating over each word pair
    
    #taking two words to find their similarity
    word1 = row["word1"].strip() 
    word2 = row["word2"].strip()
    
    #extracting glove-50 and glove-100embeddings for both words
    vec1,vec2=glove_data(g50,word1,word2)   
    cos_sim = cosine_sim(vec1,vec2)         #cosine similarity
    g50_sim.append(cos_sim)                 
    
    vec1,vec2=glove_data(g100,word1,word2)  
    cos_sim = cosine_sim(vec1,vec2)         #cosine similarity
    g100_sim.append(cos_sim)                
    
    
    #vec1,vec2=glove_data(g200,word1,word2)
    #cos_sim = cosine_sim(vec1,vec2)
    #g200_sim.append(cos_sim)
    
    
    #vec1,vec2=glove_data(g300,word1,word2)
    #cos_sim = cosine_sim(vec1,vec2)
    #g300_sim.append(cos_sim)

In [11]:
#saving similarity value in dataframe

glove_file = file[["word1","word2"]]
glove_file["GloVe_sim_50"] = g50_sim
glove_file["GloVe_sim_100"]= g100_sim
#glove_file["g200_sim"]= g200_sim
#glove_file["g300_sim"]= g300_sim
glove_file

,word1,word2,GloVe_sim_50,GloVe_sim_100
0,मोहब्बत,सेक्स,4.330526,3.480146
1,बाघ,बिल्ली,5.996535,5.456493
2,किताब,कागज़,6.407841,5.772484
3,कंप्यूटर,कीबोर्ड,6.321789,5.557799
4,कंप्यूटर,इंटरनेट,7.983269,6.982661
...,...,...,...,...
60,मनोविज्ञान,चिकित्सक,6.010799,4.880691
61,मनोविज्ञान,मन,3.808668,3.659042
62,मनोविज्ञान,स्वास्थ्य,5.971482,5.156013
63,मनोविज्ञान,विज्ञान,8.304849,7.450683


In [12]:
import pickle
with open('glove_sim.pkl','wb') as f:
    pickle.dump(file,f)
    f.close()

# CBow

In [13]:
#loading cbow50 and cbow100 models using Word2Vec

cbow50  = Word2Vec.load("./hi/50/cbow/hi-d50-m2-cbow.model")
cbow100 = Word2Vec.load("./hi/100/cbow/hi-d100-m2-cbow.model")
#cbow200 = Word2Vec.load("./hi/200/cbow/hi-d200-m2-cbow.model")
#cbow300 = Word2Vec.load("./hi/300/cbow/hi-d300-m2-cbow.model")

In [14]:
cbow50_sim,cbow100_sim,cbow200_sim,cbow300_sim =[],[],[],[]
for index, row in words.iterrows():    #iterating over each word pair
    
    #taking two words to find their similarity
    word1 = row["word1"].strip()
    word2 = row["word2"].strip()
    
    #extracting cbow50 and cbow100 embeddings for both words
    vec1 = cbow50.wv[word1]
    vec2 = cbow50.wv[word2]
    cos_sim = cosine_sim(vec1,vec2)   #cosine similarity
    cbow50_sim.append(cos_sim)        

    vec1 = cbow100.wv[word1]
    vec2 = cbow100.wv[word2]
    cos_sim = cosine_sim(vec1,vec2)   #cosine similarity
    cbow100_sim.append(cos_sim)
    
    #vec1 = cbow200.wv[word1]
    #vec2 = cbow200.wv[word2]
    #cos_sim = cosine_sim(vec1,vec2)
    #cbow200_sim.append(cos_sim)
    
    #vec1 = cbow300.wv[word1]
    #vec2 = cbow300.wv[word2]
    #cos_sim = cosine_sim(vec1,vec2)
    #cbow300_sim.append(cos_sim)

In [15]:
#saving similarity values in dataframe
cbow_file = file[["word1","word2"]]
cbow_file["Cbow_sim_50"] = cbow50_sim
cbow_file["Cbow_sim_100"]= cbow100_sim

#cbow_file["cbow200_sim"]= cbow200_sim
#cbow_file["cbow300_sim"]= cbow300_sim

cbow_file

,word1,word2,Cbow_sim_50,Cbow_sim_100
0,मोहब्बत,सेक्स,3.129996,2.907282
1,बाघ,बिल्ली,5.336025,4.952310
2,किताब,कागज़,5.469306,4.975152
3,कंप्यूटर,कीबोर्ड,7.682935,7.109529
4,कंप्यूटर,इंटरनेट,8.108631,7.576693
...,...,...,...,...
60,मनोविज्ञान,चिकित्सक,4.528347,3.823004
61,मनोविज्ञान,मन,2.513045,2.053753
62,मनोविज्ञान,स्वास्थ्य,4.304416,3.799627
63,मनोविज्ञान,विज्ञान,7.902854,7.578862


# Fasttext

In [16]:
#loading fasttext50 and fasttext100 models
ft50  = FastText.load("./hi/50/fasttext/hi-d50-m2-fasttext.model")
ft100 = FastText.load("./hi/100/fasttext/hi-d100-m2-fasttext.model")
#ft200 = FastText.load("./hi/200/fasttext/hi-d200-m2-fasttext.model")
#ft300 = FastText.load("./hi/300/fasttext/hi-d300-m2-fasttext.model")

In [17]:
ft50_sim,ft100_sim,ft200_sim,ft300_sim =[],[],[],[]
for index, row in words.iterrows(): #iterating over each word pair
    
    #taking two words to find their similarity
    word1 = row["word1"].strip()
    word2 = row["word2"].strip()
    
    #extracting fasttext50 and 100 embeddings 
    vec1 = ft50.wv[word1]
    vec2 = ft50.wv[word2]
    cos_sim = cosine_sim(vec1,vec2) #cosine similarity
    ft50_sim.append(cos_sim)
    
    vec1 = ft100.wv[word1]
    vec2 = ft100.wv[word2]
    cos_sim = cosine_sim(vec1,vec2) #cosine similarity
    ft100_sim.append(cos_sim)
    
    #vec1 = ft200.wv[word1]
    #vec2 = ft200.wv[word2]
    #cos_sim = cosine_sim(vec1,vec2)
    #ft200_sim.append(cos_sim)
    
    #vec1 = ft300.wv[word1]
    #vec2 = ft300.wv[word2]
    #cos_sim = cosine_sim(vec1,vec2)
    #ft300_sim.append(cos_sim)

In [18]:
#saving similarities in the dataframe

ft_file = file[["word1","word2"]]
ft_file["FastText_sim_50"] = ft50_sim
ft_file["FastText_sim_100"]= ft100_sim
#ft_file["ft200_sim"]= ft200_sim
#ft_file["ft300_sim"]= ft300_sim
ft_file

,word1,word2,FastText_sim_50,FastText_sim_100
0,मोहब्बत,सेक्स,4.112746,3.371399
1,बाघ,बिल्ली,6.974893,6.474872
2,किताब,कागज़,5.985891,5.339189
3,कंप्यूटर,कीबोर्ड,7.530078,6.908113
4,कंप्यूटर,इंटरनेट,8.131586,7.116359
...,...,...,...,...
60,मनोविज्ञान,चिकित्सक,6.241837,5.237499
61,मनोविज्ञान,मन,4.440145,3.905772
62,मनोविज्ञान,स्वास्थ्य,5.524333,5.233972
63,मनोविज्ञान,विज्ञान,8.489980,8.076863


# Skip grams

In [19]:
#loading skip grams 50 and 100 models
sg50  = Word2Vec.load("./hi/50/sg/hi-d50-m2-sg.model")
sg100 = Word2Vec.load("./hi/100/sg/hi-d100-m2-sg.model")
#sg200 = Word2Vec.load("./hi/200/cbow/hi-d200-m2-cbow.model")
#sg300 = Word2Vec.load("./hi/300/cbow/hi-d300-m2-cbow.model")

In [20]:
sg50_sim,sg100_sim,sg200_sim,sg300_sim =[],[],[],[]
for index, row in words.iterrows(): #iterating over each word pair
    
    #taking both words to find similarity
    word1 = row["word1"].strip()
    word2 = row["word2"].strip()
    
    #extracting skip grams 50 and 100 embeddings
    vec1 = sg50.wv[word1]
    vec2 = sg50.wv[word2]
    cos_sim = cosine_sim(vec1,vec2) #cosine similarity
    sg50_sim.append(cos_sim)

    vec1 = sg100.wv[word1]
    vec2 = sg100.wv[word2]
    cos_sim = cosine_sim(vec1,vec2)  #cosine similarity
    sg100_sim.append(cos_sim)
    
    #vec1 = sg200.wv[word1]
    #vec2 = sg200.wv[word2]
    #cos_sim = cosine_sim(vec1,vec2)
    #sg200_sim.append(cos_sim)
    
    #vec1 = sg300.wv[word1]
    #vec2 = sg300.wv[word2]
    #cos_sim = cosine_sim(vec1,vec2)
    #sg300_sim.append(cos_sim)

In [21]:
#saving similarity to dataframe
sg_file = file[["word1","word2"]]
sg_file["SkipGram_sim_50"] = sg50_sim
sg_file["SkipGram_sim_100"]= sg100_sim

#sg_file["sg200_sim"]= sg200_sim
#sg_file["sg300_sim"]= sg300_sim

sg_file

,word1,word2,SkipGram_sim_50,SkipGram_sim_100
0,मोहब्बत,सेक्स,4.556121,3.887343
1,बाघ,बिल्ली,6.995613,6.253620
2,किताब,कागज़,6.167409,5.756130
3,कंप्यूटर,कीबोर्ड,7.387562,6.722089
4,कंप्यूटर,इंटरनेट,8.132942,7.179461
...,...,...,...,...
60,मनोविज्ञान,चिकित्सक,6.005289,5.112590
61,मनोविज्ञान,मन,4.590188,4.022970
62,मनोविज्ञान,स्वास्थ्य,5.584522,5.202593
63,मनोविज्ञान,विज्ञान,8.348417,7.954475


In [22]:
#calculating accuracy
def accuracy_func(embed_df,pred_sim_name,threshold):
    ground_truth = "Ground_truth_"+str(threshold)
    true_similarity = file[ground_truth].to_list() #manually written true similarities
    pred_similarity = embed_df[pred_sim_name].to_list() #predicted similarity based on threshold values
    result=np.invert(np.logical_xor(true_similarity,pred_similarity)) #XNOR to find the common similarities
    count = np.count_nonzero(result) #counting ones in the result
    accuracy_value = count/len(words) #accuracy finding
    return accuracy_value

In [23]:
#determining similarities based on threshold value

#giving name to each type of embedding
glove_file.name ='GloVe'
cbow_file.name ='Cbow'
sg_file.name ='SkipGram'
ft_file.name ='FastText'
#embed_mapping={'glove':"GloVe",'cbow':}
embedding_types=[]
accuracies =[]
embedding_dfs=[glove_file,cbow_file,sg_file,ft_file]
dimensions = [50,100]

#this loop compares all threshold values with all cosine similarity values for all given embeddings and dimensions
for embed in embedding_dfs: 
    for threshold in thresholds:
        for dim in dimensions:
            emd_type = embed.name
            pred_column = emd_type+"pred_sim_"+str(threshold)+"_"+str(dim)
            target_column = emd_type+"_sim_"+str(dim)
            #saving predicted similarity labels in dataframe
            embed[pred_column] = np.where(embed[target_column] >= threshold, 1, 0)
            
            semi_df = pd.DataFrame(columns=["Word1","Word2","Similarity Score","Ground Truth similarity score","Label"])
            semi_df["Word1"]=file["word1"]
            semi_df["Word2"]=file["word2"]
            semi_df["Similarity Score"]=embed[target_column]
            semi_df["Ground Truth similarity score"]=file["similarity"]
            semi_df["Label"]=embed[pred_column]
            
            #accuracy
            accuracy_value=accuracy_func(embed,pred_column,threshold)
            embedding_types.append(emd_type+"_"+str(threshold)+"_"+str(dim))
            accuracies.append(accuracy_value)
            
            #save in csv file
            acc_df=pd.DataFrame(data=[['','Accuracy','=',accuracy_value,'']],columns=list(semi_df))
            final_df = pd.concat([semi_df,acc_df],ignore_index=True)
            final_df.to_csv("./Outputs/Q1/"+"Q1_"+emd_type+str(dim)+"_similarity_"+str(threshold)+".csv")

In [24]:
#save accuracy in dataframe
accuracy_df=pd.DataFrame()
accuracy_df["Embedding_type"]=embedding_types
accuracy_df["accuracy"]=accuracies
accuracy_df

,Embedding_type,accuracy
0,GloVe_4_50,0.846154
1,GloVe_4_100,0.800000
2,GloVe_5_50,0.723077
3,GloVe_5_100,0.630769
4,GloVe_6_50,0.615385
5,GloVe_6_100,0.476923
6,GloVe_7_50,0.584615
7,GloVe_7_100,0.553846
8,GloVe_8_50,0.815385
9,GloVe_8_100,0.830769
